<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b5164b5034e17641689c9958d0780b36f4acf806b1a1b5a84a48849a62b291f5
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-23 12:25:51
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.43 C
-------------------
Today PnL: 1.01 L (0.72%)
Current PnL: -19.05 L (-12.69%)
CY Booked + Current PnL: -6.95 L (-4.63%)
-------------------
Total profit:  2.37 L
Total loss:  -21.42 L
-------------------
Total Booked + Current PnL: 20.03 L (16.27%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.48%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 86.24 L (60.47%)
Deployed:  1.23 C
Current:  1.43 C
CAGR/XIRR %: 8.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.0,-8.78,73.0,X-LC,3.06,228774.0,18408.0,10363.0,0.09,8.75,4.53,13.68,37.0,1.78,1.61,26.72,XY25,NTT,REFINERIES
85,VOLTAS,1530.0,2.98,60.0,X-MC,2.42,215910.0,24168.0,13581.0,-1.13,12.60,6.29,19.69,99.0,1.78,1.52,20.26,XY25,NTT,AC
2,ABBOTINDIA,35195.0,-8.83,46.0,X-MC,3.49,90135.0,-543.0,15449.0,-0.69,-0.60,17.14,16.44,101.0,-0.04,0.63,19.46,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,96.37,50.0,M-SC,1.90,84626.0,-16151.0,16248.0,0.20,-16.03,19.20,0.09,245.0,-0.99,0.60,11.18,OX40N,NTT,DURABLES
42,ITC,452.0,-36.66,65.0,X-LC,2.12,204500.0,4362.0,16524.0,1.30,2.18,8.08,10.44,4.0,0.26,1.44,7.86,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BSOFT,836.99,-14.41,62.0,H-SC,15.19,96135.0,-44518.0,117294.0,7.39,-31.65,122.01,51.74,131.0,-0.38,0.68,11.23,XR,ATH,IT
82,VAIBHAVGBL,521.00,55.37,61.0,H-SC,4.58,134497.0,-48278.0,159352.0,6.01,-26.41,118.48,60.77,125.0,-0.30,0.95,20.46,XR,NTT,JEWELLERY
40,INFY,2275.00,-15.65,61.0,X-LC,7.35,327935.0,15047.0,156655.0,4.56,4.81,47.77,54.87,3.0,0.10,2.31,11.78,X40,BTT,IT
68,SONACOMS,806.63,-32.46,71.0,M-SC,10.22,84105.0,-17054.0,57057.0,4.04,-16.86,67.84,39.54,202.0,-0.30,0.59,18.60,AR,ATH,AUTO
30,HCLTECH,1922.01,6.24,70.0,X-LC,7.56,242424.0,500.0,59321.0,3.91,0.21,24.47,24.73,8.0,0.01,1.71,15.47,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-41.63,46.0,H-SC,18.19,88480.0,-14020.0,54619.0,-2.60,-13.68,61.73,39.61,114.0,-0.26,0.62,8.06,AR,ATH,MISC
72,TANLA,1963.11,-28.95,34.0,H-SC,17.09,190944.0,-79817.0,409766.0,-2.12,-29.48,214.60,121.86,133.0,-0.19,1.34,49.37,AR,ATH,IT
70,SURYODAY,240.00,64.47,59.0,H-SC,12.47,151327.0,-27744.0,93475.0,-1.79,-15.49,61.77,36.71,135.0,-0.30,1.07,50.15,XR,NTT,BANKS
65,SHALBY,327.00,1156.78,51.0,M-SC,22.60,171032.0,-10111.0,55585.0,-1.71,-5.58,32.50,25.10,235.0,-0.18,1.20,37.07,XY24,NTT,HEALTHCARE
46,KPIGREEN,731.05,5.18,63.0,H-SC,5.28,125496.0,199.0,58732.0,-1.41,0.16,46.80,47.03,141.0,0.00,0.88,57.13,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,41.94,47.0,M-MC,10.29,223481.0,-1481.0,169130.0,-0.13,-0.66,75.68,74.53,192.0,-0.01,1.57,31.38,XY24,BTT,STEEL
46,KPIGREEN,731.05,5.18,63.0,H-SC,5.28,125496.0,199.0,58732.0,-1.41,0.16,46.80,47.03,141.0,0.00,0.88,57.13,MH,ATH,POWER
37,INDIAMART,4810.62,-55.39,42.0,H-SC,7.36,124444.0,1108.0,130517.0,-1.21,0.90,104.88,106.72,119.0,0.01,0.88,24.49,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,121.70,68.0,H-SC,14.42,140103.0,-2376.0,21898.0,3.28,-1.67,15.63,13.70,163.0,-0.11,0.99,65.36,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.69,62.0,H-SC,2.20,229779.0,-39888.0,76218.0,-0.60,-14.79,33.17,13.47,138.0,-0.52,1.62,16.84,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,96.37,50.0,M-SC,1.90,84626.0,-16151.0,16248.0,0.20,-16.03,19.20,0.09,245.0,-0.99,0.60,11.18,OX40N,NTT,DURABLES
67,SIS,528.0,2083.94,54.0,H-SC,3.42,88141.0,-22891.0,46503.0,0.32,-20.62,52.76,21.26,156.0,-0.49,0.62,19.09,OX40N,NTT,MISC
73,TATAELXSI,9161.0,-23.71,49.0,H-MC,7.62,103303.0,-28532.0,70752.0,1.54,-21.64,68.49,32.03,98.0,-0.40,0.73,15.71,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,47.87,53.0,M-LC,5.84,207900.0,4730.0,37401.0,0.23,2.33,17.99,20.74,55.0,0.13,1.46,8.00,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.16,51.0,M-LC,10.63,210083.0,5007.0,107562.0,-0.19,2.44,51.20,54.89,52.0,0.05,1.48,27.13,X5K,ATH,METALS
46,KPIGREEN,731.05,5.18,63.0,H-SC,5.28,125496.0,199.0,58732.0,-1.41,0.16,46.80,47.03,141.0,0.00,0.88,57.13,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,62.0,M-LC,9.03,163466.0,22626.0,21872.0,1.49,16.06,13.38,31.59,66.0,1.03,1.15,46.81,XY24,NTT,BANKS
87,WIPRO,420.00,-12.89,52.0,M-LC,6.22,153289.0,2344.0,107118.0,2.44,1.55,69.88,72.51,53.0,0.02,1.08,7.67,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-21.74,66.0,H-MC,13.03,200340.0,20532.0,91876.0,0.38,11.42,45.86,62.52,88.0,0.22,1.41,49.40,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-38.01,68.0,M-SC,3.98,104700.0,13202.0,65553.0,-0.11,14.43,62.61,86.07,223.0,0.20,0.74,52.04,XR,NTT,DURABLES
32,HINDZINC,730.22,29.16,51.0,M-LC,10.63,210083.0,5007.0,107562.0,-0.19,2.44,51.20,54.89,52.0,0.05,1.48,27.13,X5K,ATH,METALS
87,WIPRO,420.00,-12.89,52.0,M-LC,6.22,153289.0,2344.0,107118.0,2.44,1.55,69.88,72.51,53.0,0.02,1.08,7.67,XR,NTT,IT
37,INDIAMART,4810.62,-55.39,42.0,H-SC,7.36,124444.0,1108.0,130517.0,-1.21,0.90,104.88,106.72,119.0,0.01,0.88,24.49,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,17.0,X-LC,13.39,148086.0,-128437.0,238507.0,1.52,-46.45,161.06,39.81,54.0,-0.54,1.04,4.38,XY24,NTT,AUTO
53,QUESS,986.00,-47.66,37.0,X-SC,38.75,51892.0,-13114.0,164041.0,-0.75,-20.17,316.12,232.18,198.0,-0.08,0.37,1.22,XY24,NTT,MISC
24,DMART,5391.45,-15.68,40.0,X-LC,10.19,103176.0,590.0,26217.0,0.57,0.57,25.41,26.13,38.0,0.02,0.73,26.29,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,-8.83,46.0,X-MC,3.49,90135.0,-543.0,15449.0,-0.69,-0.60,17.14,16.44,101.0,-0.04,0.63,19.46,X40,ATH,PHARMA
33,HONAUT,58357.33,-23.00,46.0,X-SC,2.54,108585.0,-19353.0,66487.0,-0.73,-15.13,61.23,36.84,143.0,-0.29,0.76,11.16,X40N,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.97,49.0,H-LC,0.67,157400.0,-28695.0,76182.0,1.45,-15.42,48.40,25.51,15.0,-0.38,1.11,15.99,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,65.0,X-LC,2.12,204500.0,4362.0,16524.0,1.30,2.18,8.08,10.44,4.0,0.26,1.44,7.86,X40,NTT,FMCG
85,VOLTAS,1530.00,2.98,60.0,X-MC,2.42,215910.0,24168.0,13581.0,-1.13,12.60,6.29,19.69,99.0,1.78,1.52,20.26,XY25,NTT,AC
29,HAVELLS,2069.16,-5.05,50.0,X-MC,2.77,235373.0,-12358.0,87418.0,1.66,-4.99,37.14,30.30,92.0,-0.14,1.66,7.64,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-8.78,73.0,X-LC,3.06,228774.0,18408.0,10363.0,0.09,8.75,4.53,13.68,37.0,1.78,1.61,26.72,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,37.0,X-SC,38.75,51892.0,-13114.0,164041.0,-0.75,-20.17,316.12,232.18,198.0,-0.08,0.37,1.22,XY24,NTT,MISC
3,ACC,3906.00,-38.07,47.0,X-MC,3.03,183950.0,-54001.0,206649.0,-0.51,-22.69,112.34,64.15,174.0,-0.26,1.29,3.19,XY24,BTT,CEMENT
74,TATAMOTORS,1065.00,-53.17,17.0,X-LC,13.39,148086.0,-128437.0,238507.0,1.52,-46.45,161.06,39.81,54.0,-0.54,1.04,4.38,XY24,NTT,AUTO
10,BAJAJHFL,181.50,-14.06,48.0,X-MC,5.66,181302.0,-19204.0,116360.0,-0.14,-9.58,64.18,48.45,90.0,-0.17,1.28,4.55,X40N,ATH,FINANCE
51,PAGEIND,51769.93,-25.84,47.0,X-MC,6.66,83780.0,270.0,19764.0,0.73,0.32,23.59,23.98,82.0,0.01,0.59,6.16,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,37.0,X-SC,38.75,51892.0,-13114.0,164041.0,-0.75,-20.17,316.12,232.18,198.0,-0.08,0.37,1.22,XY24,NTT,MISC
51,PAGEIND,51769.93,-25.84,47.0,X-MC,6.66,83780.0,270.0,19764.0,0.73,0.32,23.59,23.98,82.0,0.01,0.59,6.16,X40,ATH,APPARELS
57,RELAXO,1176.00,-38.81,47.0,X-SC,4.74,83378.0,-61782.0,137707.0,0.20,-42.56,165.16,52.31,136.0,-0.45,0.59,11.61,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-8.83,46.0,X-MC,3.49,90135.0,-543.0,15449.0,-0.69,-0.60,17.14,16.44,101.0,-0.04,0.63,19.46,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-34.87,49.0,X-SC,4.29,97417.0,-31253.0,76550.0,0.70,-24.29,78.58,35.20,219.0,-0.41,0.69,11.81,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.08,60.0,X-LC,12.51,290977.0,-54979.0,121687.0,2.95,-15.89,41.82,19.28,1.0,-0.45,2.05,7.57,X40,ATH,IT
40,INFY,2275.00,-15.65,61.0,X-LC,7.35,327935.0,15047.0,156655.0,4.56,4.81,47.77,54.87,3.0,0.10,2.31,11.78,X40,BTT,IT
42,ITC,452.00,-36.66,65.0,X-LC,2.12,204500.0,4362.0,16524.0,1.30,2.18,8.08,10.44,4.0,0.26,1.44,7.86,X40,NTT,FMCG
84,VBL,671.64,-16.58,60.0,X-LC,10.09,300530.0,-15312.0,127966.0,2.52,-4.85,42.58,35.67,5.0,-0.12,2.12,8.33,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,52.0,H-LC,6.49,250920.0,-10699.0,129901.0,-0.24,-4.09,51.77,45.57,7.0,-0.08,1.77,6.77,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7211.11,70.0,L-SC,6.99,80238.0,-13572.0,90188.0,0.05,-14.47,112.40,81.67,269.0,-0.15,0.56,55.38,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,26.66,81792.0,-31757.0,71805.0,-1.05,-27.97,87.79,35.27,268.0,-0.44,0.58,100.19,XR,NTT,HOTELS
49,MASFIN,398.61,-17.34,57.0,H-SC,9.56,93810.0,-4170.0,25770.0,1.49,-4.26,27.47,22.05,152.0,-0.16,0.66,36.36,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-38.01,68.0,M-SC,3.98,104700.0,13202.0,65553.0,-0.11,14.43,62.61,86.07,223.0,0.20,0.74,52.04,XR,NTT,DURABLES
46,KPIGREEN,731.05,5.18,63.0,H-SC,5.28,125496.0,199.0,58732.0,-1.41,0.16,46.80,47.03,141.0,0.00,0.88,57.13,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-190.91,75.0,M-SC,22.61,157266.0,8046.0,136129.0,-0.22,5.39,86.56,96.62,208.0,0.06,1.11,70.20,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,121.70,68.0,H-SC,14.42,140103.0,-2376.0,21898.0,3.28,-1.67,15.63,13.70,163.0,-0.11,0.99,65.36,OX40N,NTT,BANKS
68,SONACOMS,806.63,-32.46,71.0,M-SC,10.22,84105.0,-17054.0,57057.0,4.04,-16.86,67.84,39.54,202.0,-0.30,0.59,18.60,AR,ATH,AUTO
5,ANGELONE,3033.00,16.84,71.0,X-SC,7.02,206550.0,15544.0,42157.0,0.84,8.14,20.41,30.21,157.0,0.37,1.45,30.72,X40N,NTT,FINANCE
12,BANKINDIA,190.00,-21.74,66.0,H-MC,13.03,200340.0,20532.0,91876.0,0.38,11.42,45.86,62.52,88.0,0.22,1.41,49.40,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.60
1,25,44.04
2,50,74.81


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.27
LC    32.98
MC    22.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.27
X40      16.12
XY25     12.33
XR       11.19
X40N      9.36
AR        7.91
OX40N     7.60
X200      1.79
X5K       1.48
SR        1.09
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.49
X-LC    22.11
X-MC    17.11
M-SC    12.67
M-LC     5.17
H-LC     4.67
X-SC     4.63
H-MC     3.79
M-MC     1.57
L-SC     1.48
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.86,-2.17,35.52
IT,12.79,-15.75,78.22
FINANCE,12.15,-9.56,59.22
MISC,6.87,-17.91,83.24
BANKS,6.54,-7.44,63.96
PAINTS,5.63,-15.36,32.55
ELECTRICAL,5.30,-9.45,47.85
INSURANCE,3.91,0.42,36.71
AUTO,2.82,-43.47,103.34


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3272482.0,22
XR,1320563.0,14
AR,1170777.0,9
X40,756632.0,12
X40N,601658.0,8
OX40N,567801.0,10
XY25,431405.0,8
SR,266108.0,2
X5K,107562.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3379271.0,25
M-SC,1206448.0,15
X-MC,1104942.0,13
X-LC,1000216.0,13
X-SC,524090.0,6
H-MC,341331.0,3
H-LC,276775.0,3
M-LC,273953.0,4
L-SC,250330.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1161667.0      6
           AR         841303.0      5
           XR         787841.0      7
M-SC       XY24       689405.0      6
X-MC       XY24       584393.0      4
X-LC       X40        435953.0      5
           XY24       303271.0      2
X-SC       X40N       283499.0      4
H-SC       SR         266108.0      2
           OX40N      263620.0      4
X-MC       X40        244129.0      6
H-LC       AR         206083.0      2
X-LC       X40N       201799.0      3
H-MC       XY24       178703.0      1
M-MC       XY24       169130.0      1
X-SC       XY24       164041.0      1
L-SC       XR         161993.0      2
X-MC       XY25       160060.0      2
M-SC       OX40N      145092.0      4
           XY25       136129.0      1
           AR         123391.0      2
X-MC       X40N       116360.0      1
M-SC       XR         112431.0      2
M-LC       X5K        107562.0      1
           XR         107118.0      1
H-MC       XR          91876.0      1
L-SC       OX40N       88337.0      1
X-SC       X40         76550.0      1
H-MC       OX40N       70752.0      1
H-LC       X200        70692.0      1
L-MC       XR          59304.0      1
X-LC       XY25        59193.0      3
H-SC       MH          58732.0      1
L-LC       XY25        38622.0      1
M-LC       XY25        37401.0      1
           XY24        21872.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
